In [1]:
%tensorflow_version 1.6
import os, sys
from PIL import Image
import numpy as np
import keras
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers
from keras.models import Model, Sequential
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Input, BatchNormalization
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.core import Reshape, Dense, Dropout, Flatten

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.6`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = "/content/drive/My Drive/ClarenceThomasPics/"
dirs = os.listdir( path )
newPath = '/content/drive/My Drive/ClarenceThomasResized/'

def resize():
  counter = 0
  for item in dirs:
      if os.path.isfile(path+item) and item.endswith(".jpg"):
        im = Image.open(path+item)
        f, e = os.path.splitext(path+item)
        imResize = im.resize((200,200), Image.ANTIALIAS)
        imResize.save(newPath + str(counter) + 'resized.jpg', 'JPEG', quality=90)
        counter += 1
resize()

In [4]:
directory = '/content/drive/My Drive/ClarenceThomasNoColor/'
for filename in os.listdir(newPath):
  if filename.endswith(".jpg"):
    img = Image.open(newPath+filename).convert('LA')
    img.save(directory+filename[:-3]+'png')

In [5]:
X_train = []
for filename in os.listdir(directory)[:-100]:
  print(filename)
  if filename.endswith(".png"):
    img = Image.open(directory+filename)
    imgArray = np.array(img)
    fileTemp = []
    for row in imgArray:
      rowTemp = []
      for pix in row:
        rowTemp.append(pix[0])
      fileTemp.append(rowTemp)
    X_train.append(fileTemp)

0resized.png
14resized.png
29resized.png
44resized.png
59resized.png
1resized.png
15resized.png
30resized.png
45resized.png
60resized.png
2resized.png
16resized.png
31resized.png
46resized.png
61resized.png
3resized.png
17resized.png
32resized.png
47resized.png
62resized.png
4resized.png
18resized.png
33resized.png
48resized.png
63resized.png
5resized.png
19resized.png
34resized.png
49resized.png
64resized.png
20resized.png
6resized.png
35resized.png
50resized.png
65resized.png
21resized.png
7resized.png
36resized.png
51resized.png
66resized.png
22resized.png
8resized.png
37resized.png
74resized.png
52resized.png
67resized.png
23resized.png
89resized.png
9resized.png
38resized.png
75resized.png
53resized.png
68resized.png
24resized.png
90resized.png
10resized.png
39resized.png
76resized.png
54resized.png
69resized.png
25resized.png
40resized.png
104resized.png
11resized.png
77resized.png
91resized.png
119resized.png
55resized.png
70resized.png
26resized.png
105resized.png
41resized.png

In [6]:
X_train=np.array([[[coord for coord in xk] for xk in xj] for xj in X_train], ndmin=3) #this case for N=3

In [7]:
print(X_train)
print(type(X_train[0]))
print(type(X_train[0][0]))
print(X_train.shape)

[[[ 88  88  89 ...  16  15  12]
  [ 86  86  86 ...  14  15  14]
  [ 84  85  85 ...  12  14  15]
  ...
  [108 107 107 ... 179 148 166]
  [101 101 101 ... 161 157 186]
  [ 96  96  97 ... 185 199 184]]

 [[ 13  15  17 ...  15  15  15]
  [ 13  16  18 ...  15  15  15]
  [ 14  16  19 ...  15  15  14]
  ...
  [ 13  10   8 ...   6   8   9]
  [ 20  17  15 ...   4   6   7]
  [ 22  19  17 ...  26  28  29]]

 [[  4   4   4 ...  23  22  22]
  [  4   4   4 ...  23  22  22]
  [  4   4   4 ...  23  22  22]
  ...
  [  7   6   6 ...  18  10   9]
  [  7   6   6 ...   0   0   4]
  [  7   6   6 ...  14  13   1]]

 ...

 [[234 234 234 ... 128 128 127]
  [234 234 234 ... 130 129 128]
  [234 234 234 ... 131 130 130]
  ...
  [215 215 216 ...  46  47  50]
  [207 207 208 ...  42  42  46]
  [200 201 201 ...  37  36  41]]

 [[  1   1   1 ...   2   2   2]
  [  1   1   1 ...   2   2   2]
  [  1   1   1 ...   2   2   2]
  ...
  [  1   1   1 ...  25  21  34]
  [  1   1   1 ...  25  24  35]
  [  1   1   1 ...  27  28  

In [8]:
totLength = len(os.listdir(directory))
totLength

524

In [9]:
X_test = []
for filename in os.listdir(directory)[-100:]:
  tempArray = []
  if filename.endswith(".png"):
    img = Image.open(directory+filename)
    imgArray = np.asarray(img)
    for row in imgArray:
      rowTemp = []
      for pix in row:
        pix = np.ndarray.tolist(pix[:1])[0]
        rowTemp.append(pix)
      tempArray.append(rowTemp)
    X_test.append(tempArray)
  else:
    continue

In [10]:
X_test=np.array([[[coord for coord in xk] for xk in xj] for xj in X_test], ndmin=3) #this case for N=3

print(type(X_test))
print(type(X_test[0]))
print(X_test.shape)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(100, 200, 200)


In [11]:
y_train, y_test = np.asarray([1 for i in range(totLength - 100)]), np.asarray([1 for i in range(totLength-100, totLength)])
print(y_train[0])
print(type(X_train))

1
<class 'numpy.ndarray'>


In [12]:
print(tf.__version__)
keras.backend.image_data_format()
latent_dimension = 100

X_train = (X_train.astype(np.float32) - 127.5)/127.5
X_train = X_train[:, np.newaxis,:64, :64]
#X_train = X_train[:, np.newaxis,:24, :24]
print(X_train.shape)
adam = Adam(lr=0.00025, beta_1=0.7)

#Rough Draft Model
"""
generator = Sequential()
generator.add(Dense(128*6*6, input_dim=latent_dimension))
generator.add(LeakyReLU(0.175))
generator.add(Reshape((128, 6, 6)))
generator.add(BatchNormalization())
generator.add(UpSampling2D(size=(2, 2), data_format='channels_first'))
generator.add(Conv2D(64, kernel_size=(4, 4), padding='same', data_format='channels_first'))
generator.add(LeakyReLU(0.175))
generator.add(UpSampling2D(size=(2, 2), data_format='channels_first'))

generator.add(Conv2D(1, kernel_size=(4, 4), padding='same', activation='tanh', data_format='channels_first'))
print(generator.summary())
generator.compile(loss='binary_crossentropy', optimizer=adam)

# Discriminator Model
discriminator = Sequential()
discriminator.add(Conv2D(24, kernel_size=(4, 4), strides=(2, 2), padding='same', 
                         input_shape=(1, 24, 24), data_format='channels_first',
                         kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.175))
discriminator.add(Dropout(0.15))
discriminator.add(BatchNormalization())
discriminator.add(Conv2D(128, kernel_size=(4, 4), strides=(2, 2), 
                         padding='same', data_format='channels_first'))
discriminator.add(LeakyReLU(0.175))
discriminator.add(Dropout(0.15))

discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
print(discriminator.summary())
discriminator.compile(loss='binary_crossentropy', optimizer=adam)
"""


#"""
generator = Sequential()

generator.add(Dense(256*4*4, input_dim=latent_dimension, activation='relu'))
generator.add(Reshape((256, 4, 4)))
generator.add(UpSampling2D(size=(2, 2), data_format='channels_first'))

generator.add(Conv2D(256, kernel_size=(3, 3), padding='same', data_format='channels_first'))
generator.add(BatchNormalization(momentum=0.75))
generator.add(LeakyReLU(0.1))
generator.add(UpSampling2D(size=(2, 2), data_format='channels_first'))

generator.add(Conv2D(256, kernel_size=(3, 3), padding='same', data_format='channels_first'))
generator.add(BatchNormalization(momentum=0.75))
generator.add(LeakyReLU(0.1))
generator.add(UpSampling2D(size=(2, 2), data_format='channels_first'))

generator.add(Conv2D(128, kernel_size=(3, 3), padding='same', data_format='channels_first'))
generator.add(BatchNormalization(momentum=0.75))
generator.add(LeakyReLU(0.1))
generator.add(UpSampling2D(size=(2, 2), data_format='channels_first'))
generator.add(Conv2D(128, kernel_size=(3, 3), padding='same', data_format='channels_first'))

generator.add(BatchNormalization(momentum=0.75))
generator.add(LeakyReLU(0.1))


generator.add(Conv2D(1, kernel_size=(3, 3), padding='same', activation='tanh', data_format='channels_first'))
print(generator.summary())
generator.compile(loss='binary_crossentropy', optimizer=adam)

# Discriminator Model
discriminator = Sequential()
discriminator.add(Conv2D(32, kernel_size=(3, 3), strides=(2, 2), padding='same', 
                         input_shape=(1, 64, 64), data_format='channels_first',
                         kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.1))
discriminator.add(Dropout(0.1))
discriminator.add(Conv2D(64, kernel_size=(3, 3), strides=(2, 2), padding='same', data_format='channels_first'))
discriminator.add(BatchNormalization(momentum=0.75))
discriminator.add(LeakyReLU(0.1))

discriminator.add(Dropout(0.1))
discriminator.add(Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same', data_format='channels_first'))
discriminator.add(BatchNormalization(momentum=0.75))
discriminator.add(LeakyReLU(0.1))

discriminator.add(Dropout(0.1))
discriminator.add(Conv2D(256, kernel_size=(3, 3), strides=(2, 2), padding='same', data_format='channels_first'))
discriminator.add(BatchNormalization(momentum=0.75))
discriminator.add(LeakyReLU(0.1))

discriminator.add(Dropout(0.1))
discriminator.add(Conv2D(512, kernel_size=(3, 3), strides=(2, 2), padding='same', data_format='channels_first'))
discriminator.add(BatchNormalization(momentum=0.75))
discriminator.add(LeakyReLU(0.1))

discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
print(discriminator.summary())
discriminator.compile(loss='binary_crossentropy', optimizer=adam)
#"""
# Generator Model


# Adversarial Network Creation
gInput = Input(shape=(latent_dimension,))
x = generator(gInput)
discriminator.trainable = False
gOutput = discriminator(x)
gan = Model(inputs=gInput, outputs=gOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

1.15.2
(424, 1, 64, 64)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4096)              413696    
_________________________________________________________________
reshape_1 (Reshape)          (None, 256, 4, 4)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 256, 8, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 8, 8)         590080    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 8, 8)         32        
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256, 8, 8)         0         
_______________

In [13]:
def save(epoch):
    generator.save('models/generator_epoch_%d.h5' % epoch)
    discriminator.save('models/discriminator_epoch_%d.h5' % epoch)

def plots(epoch, dim=(12, 12), exs=100, figsize=(12, 12)):
    noise = np.random.normal(0, 1, size=[exs, latent_dimension])
    generatedImages = generator.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i, 0], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('images/pics_epoch_%d.png' % epoch)

In [14]:
epochs = 15
batchSize = 128
batchCount = X_train.shape[0] / batchSize
print(X_train.shape)

for epoch in range(1, epochs):
    print('Epoch %d' % epoch)
    for i in tqdm(range(int(batchCount))):
        # Creating input noise,images
        noise = np.random.normal(0, 1, size=[batchSize, latent_dimension])
        imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]
        print(imageBatch.ndim)
        
        # Creating wrong images
        generatedImages = generator.predict(noise)
        X = np.concatenate([imageBatch, generatedImages])

        # Labels both data, Label smoothing
        yDis = np.zeros(2*batchSize)
        yDis[:batchSize] = 0.9

        # Discriminator
        discriminator.trainable = True
        dloss = discriminator.train_on_batch(X, yDis)

        # Generator
        noise = np.random.normal(0, 1, size=[batchSize, latent_dimension])
        discriminator.trainable = False
        yGen = np.ones(batchSize)
        gloss = gan.train_on_batch(noise, yGen)
    #plots(epoch)
    save(epoch)


  0%|          | 0/3 [00:00<?, ?it/s]

(424, 1, 64, 64)
Epoch 1
4



 33%|███▎      | 1/3 [00:10<00:20, 10.39s/it]

4


 67%|██████▋   | 2/3 [00:10<00:07,  7.36s/it]

...

 33%|███▎      | 1/3 [00:00<00:00,  3.37it/s]

4


 67%|██████▋   | 2/3 [00:00<00:00,  3.38it/s]

4


100%|██████████| 3/3 [00:00<00:00,  3.37it/s]
